In [38]:
import requests
from urllib.parse import quote_plus as urlencode
from rdflib import Dataset, Graph, compare
from tqdm import tqdm

In [39]:
inputFile = '../data/graphs/imageRegions.trig'
endpoint = "http://localhost:7776/rdf-graph-store"

In [40]:
inputData = Dataset()

In [41]:
updateCondition = """
PREFIX crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.ics.forth.gr/isl/CRMdig/>
ASK { 
  ?s a crmdig:D35_Area ;
      crm:P33_used_specific_technique <https://github.com/swiss-art-research-net/bso-image-segmentation> .
}
"""

In [42]:
devnull = inputData.parse(inputFile, format='trig')

In [43]:
headers = {'Accept': 'text/turtle'}

graphs= {
    'new': [],
    'changed': [],
    'unchanged': []}

for context in inputData.contexts():
    if context.identifier.startswith("http"):
        r = requests.get(endpoint, headers=headers, params={"graph": context.identifier})
        if r.ok:
            remoteGraph = Graph()
            remoteGraph.parse(data=r.text, format='turtle')
            if compare.to_isomorphic(context) == compare.to_isomorphic(remoteGraph):
                graphs['unchanged'].append((context, remoteGraph))
            else:
                graphs['changed'].append((context, remoteGraph))
        else:
            graphs['new'].append((context, False))

In [44]:
print(len(graphs['new']), "new graphs")
print(len(graphs['changed']), "updated graphs")
print(len(graphs['unchanged']), "unchanged graphs")

0 new graphs
2 updated graphs
7 unchanged graphs


In [45]:
def putGraph(context, endpoint):
    data = context.serialize(format='turtle')
    params = {"graph": context.identifier}
    headers = {"Content-Type" : "text/turtle"}
    r = requests.put(endpoint, params=params, data=data, headers=headers, auth=("admin","admin"))
    return r.ok

In [46]:
graphsToUpdate = [d[0] for d in graphs['new']]

for graphPair in graphs['changed']:
    update = False
    if updateCondition:
        for result in graphPair[1].query(updateCondition):
            break
        update = result
    else:
        update = True
        
    if update:
        print("Overwriting", graphPair[0].identifier)
        graphsToUpdate.append(graphPair[0])


Overwriting https://resource.swissartresearch.net/digitalobject/b957bb50-c31c-3f6e-a98f-1fae9bf4341d/container/context


In [47]:
for g in tqdm(graphsToUpdate):
    putGraph(g, endpoint)

100%|██████████| 1/1 [00:00<00:00,  5.53it/s]


In [48]:
graphsToUpdate

[<Graph identifier=https://resource.swissartresearch.net/digitalobject/b957bb50-c31c-3f6e-a98f-1fae9bf4341d/container/context (<class 'rdflib.graph.Graph'>)>]

In [49]:
r

<Response [200]>